In [29]:
# At the beginning of the COVID-19 pandemic, Cornell University depopulated its main campus and instructed laboratories to temporarily cease lab work. 
# This presented the opportunity to shut off fume hoods to save energy. What was anticipated to be highly successful showed the need for improvements in methods. 
# This paper offers steps for fume hood hibernation and system design elements that will allow for efficient shutdown that will make the hood hibernation program more effective.

# https://pubs.acs.org/doi/10.1021/acs.chas.2c00004

In [30]:
import nbimporter
from query_v2 import fume_query, query_to_list, total_energy, total_time_sash_open
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
from dateutil import tz
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

points = pd.read_csv("pointnames.csv")

In [31]:
sash_point = points["Sash"][0]
server  = points["Server"][0]
time_start = str(datetime(2023, 1,1))
time_end = str(datetime(2023, 11,2))
sash_points = points["Sash"]

# def has_been_closed(sash_point,server,start,end):
#     query = fume_query(sash_point,server,start,end)
#     inches = [i[0] for i in query]
#     ratio = inches.count(min(inches))/len(inches)
#     return(ratio)

#ratio_list = [has_been_closed(sash_points[i], server,time_start,time_end) for i in range(len(sash_points))]
#print(ratio_list)

In [32]:
def sash_mean_sd_min_max(sash,server,start,end):
    query = fume_query(sash,server,start,end)
    sd_value = np.std(query["value"])
    mean_value = np.mean(query["value"])
    max_value = np.max(query["value"])
    min_value = np.min(query["value"])
    return [sd_value, mean_value, max_value, min_value]

big_list = [sash_mean_sd_min_max(sash_points[i], server,time_start,time_end) for i in range(len(sash_points))]



<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [39]:
#big_list

sd_list = [i[0] for i in big_list]
mean_list = [i[1] for i in big_list]
max_list = [i[2] for i in big_list]
min_list = [i[3] for i in big_list]

data = {"Name": points["Lab"], "Mean": mean_list, "Stdev": sd_list, "Min": min_list, "Max": max_list}
ndf = pd.DataFrame(data)

display(ndf)


,Name,Mean,Stdev,Min,Max
0,g54,5.557060,3.114386,0.8,17.6
1,141,2.687170,0.758633,1.6,19.3
2,143,3.354170,1.068064,1.4,21.2
3,147b,3.538636,1.831949,1.9,19.9
4,153,2.828358,1.560029,1.1,18.1
...,...,...,...,...,...
61,445b,0.046086,0.362082,0.0,3.2
62,453,3.791219,1.804626,2.0,24.9
63,453b,3.107806,1.877014,1.4,20.5
64,457,1.751237,1.379835,1.1,23.2


In [40]:
fig = px.bar(ndf, x = "Name", y = "Mean",title="Mean Sash Heights Across Labs")
#fig.write_image("C:\\Users\\Rob\\Desktop\\ESW\\Fume\\energy_across_labs.png",scale=4)
fig.show()

In [41]:
fig = px.bar(ndf, x = "Name", y = "Stdev",title="Standard Deviation of Sash Heights Across Labs")
#fig.write_image("C:\\Users\\Rob\\Desktop\\ESW\\Fume\\energy_across_labs.png",scale=4)
fig.show()

In [42]:
# ndf["Candidate for Hibernation"] = np.where((ndf["Stdev"]<0.25)&(ndf["Mean"]<0.5), True,False)
ndf["Candidate for Hibernation"] = np.where((ndf["Stdev"]<0.25), True,False)

In [43]:
ndf

,Name,Mean,Stdev,Min,Max,Candidate for Hibernation
0,g54,5.557060,3.114386,0.8,17.6,False
1,141,2.687170,0.758633,1.6,19.3,False
2,143,3.354170,1.068064,1.4,21.2,False
3,147b,3.538636,1.831949,1.9,19.9,False
4,153,2.828358,1.560029,1.1,18.1,False
...,...,...,...,...,...,...
61,445b,0.046086,0.362082,0.0,3.2,False
62,453,3.791219,1.804626,2.0,24.9,False
63,453b,3.107806,1.877014,1.4,20.5,False
64,457,1.751237,1.379835,1.1,23.2,False


In [44]:
filtered_df = ndf[ndf['Candidate for Hibernation'] == True]
print(filtered_df)

print("41 and 57 have 0 S.D and 0 mean.")

    Name      Mean     Stdev  Min   Max  Candidate for Hibernation
7    160  0.301821  0.017997  0.2   1.6                       True
9    221  1.389728  0.077900  1.3   1.7                       True
13   233  1.435752  0.068253  1.4   7.8                       True
36   333  0.942376  0.168132  0.8  17.1                       True
41  345b  0.000000  0.000000  0.0   0.0                       True
48   409 -2.545390  0.050259 -2.7  -2.5                       True
51  417b  1.101851  0.162226  1.0   2.1                       True
57  433b  0.000000  0.000000  0.0   0.0                       True
60   445  0.020374  0.149591  0.0   1.2                       True
41 and 57 have 0 S.D and 0 mean.
